In [1]:
import pymc3
import umap
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import time
import os
from scipy.stats import zscore
import threading
import pickle
%matplotlib inline

## Load in z-scored data

In [2]:
df = pickle.load(open("../data/processed/fc_z.p","rb"))
df.head()

,Cell Size,Cell Circularity,Cell Aspect Ratio,Cell Tracker Intensity,PI Intensity,AnexinV Intensity
0_2G3,62.4662,-9.764209,2.257279,-0.613713,-0.912228,-1.053166
1_2G3,-0.327282,-9.343066,84.071640,-0.941838,-0.915353,-1.213429
2_2G3,-0.590511,-7.180160,15.145722,-0.891318,-0.914065,-1.198231
3_2G3,-0.531419,-8.459477,30.618888,-0.818131,-0.911537,-1.199510
4_2G3,-0.644231,0.652443,1.079209,-0.827969,-0.913367,-1.194802


## Choose
a) n_neighbors - UMAP parameter similar to perplexity in t-sne   
b) min_distance - UMAP parameter reflecting how far apart points can be  
b) n_subsample - number of samples to use from each time-stimulus-genotype combination  
c) n_iter - number of times to run the umap subsample  

In [3]:
n_neighbors_l = [3, 5, 10, 15, 100]
min_distance_l = [0.1, 0]
n_subsample = 100
n_iter = 3

In [4]:
meta_df = pd.read_csv("../data/meta.tsv",sep="\t",index_col=0)

In [5]:
meta_df

,Timepoint,Stimuli,Sample,Genotype
0_2G3,3,G,2,WT
1_2G3,3,G,2,WT
2_2G3,3,G,2,WT
3_2G3,3,G,2,WT
4_2G3,3,G,2,WT
...,...,...,...,...
4592_4B46,46,B,4,WT
4593_4B46,46,B,4,WT
4594_4B46,46,B,4,WT
4595_4B46,46,B,4,WT


In [6]:
df = pd.concat((df,meta_df),axis=1)
df.head()

,Cell Size,Cell Circularity,Cell Aspect Ratio,Cell Tracker Intensity,PI Intensity,AnexinV Intensity,Timepoint,Stimuli,Sample,Genotype
0_2G3,62.4662,-9.764209,2.257279,-0.613713,-0.912228,-1.053166,3,G,2,WT
1_2G3,-0.327282,-9.343066,84.071640,-0.941838,-0.915353,-1.213429,3,G,2,WT
2_2G3,-0.590511,-7.180160,15.145722,-0.891318,-0.914065,-1.198231,3,G,2,WT
3_2G3,-0.531419,-8.459477,30.618888,-0.818131,-0.911537,-1.199510,3,G,2,WT
4_2G3,-0.644231,0.652443,1.079209,-0.827969,-0.913367,-1.194802,3,G,2,WT


## Run umap

## Loop through and run umap
### Use 2 cpus?

In [7]:
attrs = ["Cell Size", "Cell Circularity", "Cell Aspect Ratio", 
         "Cell Tracker Intensity", "PI Intensity", "AnexinV Intensity"]

In [8]:
def run_umap(df, f_save, n_neighbors, min_distance, subsample, attrs):
    t1 = time.time()
    reducer = umap.UMAP(n_neighbors=n_neighbors, min_dist=min_distance)
    embedding = reducer.fit_transform(df[attrs])
    embedding.shape
    t2 = time.time()
    print(f"Time in minutes:{(t2-t1)/60}")

    pickle.dump([embedding,df.index],open(f_save.replace(".p","") + ".p",'wb'))
    with open(f_save.replace(".p","") + "_time_minutes.txt","w") as f:
        f.write(str((t2-t1)/60))
    return

In [10]:
for i in range(n_iter):
    print('i',i)
    for neigh in n_neighbors_l:
        print('number of neighbors', neigh)
        for dist in min_distance_l:
            print('minimum distance', dist)
            # File name
            curr_f_save = f"../results/umap_out/embedding_{i}_{neigh}_{dist}.p" #umap_results
            # Collect samples
            samples = df.groupby(["Stimuli","Genotype","Timepoint"]).apply(lambda x: x.sample(n=n_subsample).reset_index())
            samples = samples.set_index("index")
            run_umap(samples, curr_f_save, neigh, dist, n_subsample, attrs)

i 0
number of neighbors 3
minimum distance 0.1


/home/isshamie/software/anaconda2/envs/umap/lib/python3.7/site-packages/umap/nndescent.py:92: NumbaPerformanceWarning: 
The keyword argument 'parallel=True' was specified but no transformation for parallel execution was possible.

To find out why, try turning on parallel diagnostics, see http://numba.pydata.org/numba-doc/latest/user/parallel.html#diagnostics for help.

File "../../../../home/isshamie/software/anaconda2/envs/umap/lib/python3.7/site-packages/umap/utils.py", line 409:
@numba.njit(parallel=True)
def build_candidates(current_graph, n_vertices, n_neighbors, max_candidates, rng_state):
^

  current_graph, n_vertices, n_neighbors, max_candidates, rng_state
/home/isshamie/software/anaconda2/envs/umap/lib/python3.7/site-packages/numba/compiler.py:602: NumbaPerformanceWarning: 
The keyword argument 'parallel=True' was specified but no transformation for parallel execution was possible.

To find out why, try turning on parallel diagnostics, see http://numba.pydata.org/numba-doc/la

Time in minutes:0.9249130368232727
minimum distance 0


/home/isshamie/software/anaconda2/envs/umap/lib/python3.7/site-packages/numba/compiler.py:602: NumbaPerformanceWarning: 
The keyword argument 'parallel=True' was specified but no transformation for parallel execution was possible.

To find out why, try turning on parallel diagnostics, see http://numba.pydata.org/numba-doc/latest/user/parallel.html#diagnostics for help.

File "../../../../home/isshamie/software/anaconda2/envs/umap/lib/python3.7/site-packages/umap/nndescent.py", line 47:
    @numba.njit(parallel=True)
    def nn_descent(
    ^

  self.func_ir.loc))
/home/isshamie/software/anaconda2/envs/umap/lib/python3.7/site-packages/umap/spectral.py:229: UserWarning: Embedding a total of 145 separate connected components using meta-embedding (experimental)
  n_components
/home/isshamie/software/anaconda2/envs/umap/lib/python3.7/site-packages/sklearn/manifold/spectral_embedding_.py:235: UserWarning: Graph is not fully connected, spectral embedding may not work as expected.
  warnings.w

Time in minutes:0.9227564970652262
number of neighbors 5
minimum distance 0.1


...

Time in minutes:1.6066852966944376
number of neighbors 100
minimum distance 0.1


/home/isshamie/software/anaconda2/envs/umap/lib/python3.7/site-packages/numba/compiler.py:602: NumbaPerformanceWarning: 
The keyword argument 'parallel=True' was specified but no transformation for parallel execution was possible.

To find out why, try turning on parallel diagnostics, see http://numba.pydata.org/numba-doc/latest/user/parallel.html#diagnostics for help.

File "../../../../home/isshamie/software/anaconda2/envs/umap/lib/python3.7/site-packages/umap/nndescent.py", line 47:
    @numba.njit(parallel=True)
    def nn_descent(
    ^

  self.func_ir.loc))


Time in minutes:3.893318402767181
minimum distance 0


/home/isshamie/software/anaconda2/envs/umap/lib/python3.7/site-packages/numba/compiler.py:602: NumbaPerformanceWarning: 
The keyword argument 'parallel=True' was specified but no transformation for parallel execution was possible.

To find out why, try turning on parallel diagnostics, see http://numba.pydata.org/numba-doc/latest/user/parallel.html#diagnostics for help.

File "../../../../home/isshamie/software/anaconda2/envs/umap/lib/python3.7/site-packages/umap/nndescent.py", line 47:
    @numba.njit(parallel=True)
    def nn_descent(
    ^

  self.func_ir.loc))


Time in minutes:3.9242709279060364


In [4]:
# t1 = time.time()
# reducer = umap.UMAP()
# embedding = reducer.fit_transform(df_short)
# embedding.shape
# t2 = time.time()
# print(f"Time in seconds:{(t2-t1)/60}")
# with open("time_took.txt","w") as f:
#     f.write(str(t2-t1))

In [5]:
def run_umap(df,f_save):
    #df = pickle.load(open(in_f,"rb"))
    t1 = time.time()
    reducer = umap.UMAP()
    embedding = reducer.fit_transform(df.iloc[:,:-1])
    embedding.shape
    t2 = time.time()
    print(f"Time in minutes:{(t2-t1)/60}")

    pickle.dump(embedding,open(f_save,'wb'))
    with open(f_save + "_time_minutes.txt","w") as f:
        f.write(str((t2-t1)/60))
    return

In [6]:
maximumNumberOfThreads = 32
threadLimiter = threading.BoundedSemaphore(maximumNumberOfThreads)

class EncodeThread(threading.Thread):
    def run(self):
        threadLimiter.acquire()
        try:
            run_umap(df=df,f_save="umap_results.p")
        finally:
            threadLimiter.release()

In [7]:
EncodeThread().run()

/home/isshamie/software/anaconda2/envs/umap/lib/python3.7/site-packages/numba/compiler.py:602: NumbaPerformanceWarning: 
The keyword argument 'parallel=True' was specified but no transformation for parallel execution was possible.

To find out why, try turning on parallel diagnostics, see http://numba.pydata.org/numba-doc/latest/user/parallel.html#diagnostics for help.

File "../../../home/isshamie/software/anaconda2/envs/umap/lib/python3.7/site-packages/umap/rp_tree.py", line 135:
@numba.njit(fastmath=True, nogil=True, parallel=True)
def euclidean_random_projection_split(data, indices, rng_state):
^

  self.func_ir.loc))
/home/isshamie/software/anaconda2/envs/umap/lib/python3.7/site-packages/umap/nndescent.py:92: NumbaPerformanceWarning: 
The keyword argument 'parallel=True' was specified but no transformation for parallel execution was possible.

To find out why, try turning on parallel diagnostics, see http://numba.pydata.org/numba-doc/latest/user/parallel.html#diagnostics for help.

Time in minutes:1238.94629402558


## Save values

In [9]:
embedding = pickle.load(open("umap_results.p","rb"))
embedding


array([[ 14.301497 ,  -5.806734 ],
       [ -0.9557591, -15.899615 ],
       [ 11.957989 ,   9.023507 ],
       ...,
       [ -1.963811 ,  -0.5268107],
       [  9.15024  ,  -1.1823806],
       [ -9.841979 ,   0.8918857]], dtype=float32)

In [10]:
embedding.shape

(25577941, 2)